In [1]:
%config InlineBackend.figure_format = "svg"

import sys
sys.path.append("../")

from src.networks.ddn import DynamicDecisionNetwork as DDN
from src.networks.nodes import DiscreteNode

# Creating a dynamic decision network

## Example: Tiger problem

### Constructing the Dynamic Decision Network

In [2]:
# Create nodes
s0 = DiscreteNode(("S", 0), node_type=DDN.state_type, value_space=[0, 1])
a0 = DiscreteNode(("A", 0), node_type=DDN.action_type, value_space=[0, 1, 2])
s1 = DiscreteNode(("S", 1), node_type=DDN.state_type, value_space=[0, 1])
o1 = DiscreteNode(("O", 1), node_type=DDN.observation_type, value_space=[0, 2])
r1 = DiscreteNode(("R", 1), node_type=DDN.reward_type, value_space=[-100, -1, 10])

# Discount factor of the POMDP
discount = 0.8

# Create the initial DDN structure
ddn = DDN(discount=discount)
ddn.add_nodes([s0, s1, o1, r1, a0])
ddn.add_edges([
    (("S", 0), ("S", 1)), 
    (("A", 0), ("S", 1)), 
    (("A", 0), ("R", 1)), 
    (("A", 0), ("O", 1)), 
    (("S", 1), ("O", 1)), 
    (("S", 1), ("R", 1)) 
])
ddn.draw()

### Adding CPT data to the DDN

In [3]:
# Add data for node State 0
data = {
    ("S", 0): [0,1], 
    "Prob": [0.5,0.5]
}
ddn.add_pt(("S", 0), data)

# Add data for node State 1
data = {
    ("S", 0): [0,0,0,0,0,0,1,1,1,1,1,1], 
    ("A", 0): [0,0,1,1,2,2,0,0,1,1,2,2], 
    ("S", 1): [0,1,0,1,0,1,0,1,0,1,0,1], 
    "Prob": [1,0,0.5,0.5,0.5,0.5,0,1,0.5,0.5,0.5,0.5]
}
ddn.add_pt(("S", 1), data)

# Add data for node Evidence
data = {
    ("A", 0): [0,0,0,0,1,1,1,1,2,2,2,2], 
    ("S", 1): [0,0,1,1,0,0,1,1,0,0,1,1], 
    ("O", 1): [0,1,0,1,0,1,0,1,0,1,0,1], 
    "Prob": [0.85,0.15,0.15,0.85,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5]
}
ddn.add_pt(("O", 1), data)

# Add data for node Reward
data = {
    ("A", 0): [0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2,2], 
    ("S", 1): [0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,1], 
    ("R", 1): [-100,-1,10,-100,-1,10,-100,-1,10,-100,-1,10,-100,-1,10,-100,-1,10], 
    "Prob": [0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,1,0,0]
}
ddn.add_pt(("R", 1), data)

# Initialize ddn
ddn.initialize()

In [4]:
from src.utils import belief_update

# Check belief-updating
actions = {("A", 0): 1}
observations = {("O", 1): 1}
belief_update(ddn, ddn.get_belief_state(), actions, observations, n_samples=100)

Got query nodes.
Queried new belief.
Replaced column names
Finished belief updating.


{('S',
  0):    (S, 0)  Prob
 0     0.0  0.54
 1     1.0  0.46}

### Calculating best action

In [5]:
from src.rl_algorithms.pomdp_lookahead import build_tree, lookahead_policy, q_value
from src.trees.tree import Tree

In [6]:
# Build the lookahead tree
horizon = 2
action_space = ddn.get_space(ddn.action_type)
observation_space = ddn.get_space(ddn.observation_type)
tree = build_tree({}, action_space, observation_space, horizon)

In [7]:
tree.children[0].children[0].children[0].get_attribute("type")

'action'

In [8]:
n_samples = 100
value = q_value(ddn, tree.children[0], ddn.get_belief_state(), n_samples)
print(value)

Got query nodes.
Queried new belief.
Replaced column names
Finished belief updating.
Got query nodes.


KeyboardInterrupt: 